In [7]:
library(dplyr)
library(lsa)
library(DOT)
library(Seurat)
set.seed(42) 

In [8]:
# table output folder
out_folder='tab_out'
if (!dir.exists(out_folder)) {
  dir.create(out_folder, recursive = TRUE)
} else {
  cat("Folder already exists:", out_folder, "\n")
}

Folder already exists: tab_out 


In [9]:
corr_cos_plot <- function(X_org_, X_pred_, sample = TRUE) {
  # Transpose the data if sample is TRUE
  if (!sample) {
    X_pred_ <- t(X_pred_)
    X_org_ <- t(X_org_)
  }
  
  # Compute the correlation matrix
  corr_matrix <- cor(X_org_, X_pred_, use = "pairwise.complete.obs")
  num_cols <- ncol(X_org_)
  correlations <- diag(corr_matrix)
  
  # Create a named vector for correlations
  correlation_results <- setNames(correlations, colnames(X_org_))
  mean_corr <- mean(correlation_results)
  
  # Compute cosine similarity
cosine_results <- sapply(seq_len(ncol(X_org_)), function(i) {
    cosine(X_org_[, i], X_pred_[, i])
  })
  mean_cos <- mean(cosine_results, na.rm = TRUE)
    
  # Return the results
  return(list(correlation_results = correlation_results, cosine_results = cosine_results))
}


# Load scRNAseq reference data with cell annotation

In [10]:
metadata=read.csv('../tab/sc_metadata.csv',row.names = 1)
head(metadata)

# Define sample paths
samples <- list(
  BDP1083 = "/data/lemsaraa/amina/ST/ourData/sc/06092024/cellranger/BDP1083_filtered_seurat_comp.h5",
  BDP1105 = "/data/lemsaraa/amina/ST/ourData/sc/06092024/cellranger/BDP1105_filtered_seurat_comp.h5",
  BDP1130 = "/data/lemsaraa/amina/ST/ourData/sc/06092024/cellranger/BDP1130_filtered_seurat_comp.h5",
  BDP1131 = "/data/lemsaraa/amina/ST/ourData/sc/06092024/cellranger/BDP1131_filtered_seurat_comp.h5"
)

# Create an empty list to store Seurat objects
seurat_objects <- list()

# Loop through each sample and read the H5 file into a Seurat object
for (sample_id in names(samples)) {
  cat("Loading sample:", sample_id, "\n")
  
  # Convert H5 file to Seurat object
  counts <- Read10X_h5(samples[[sample_id]])
  
  # Create a Seurat object
  seurat_obj <- CreateSeuratObject(counts = counts, project = sample_id)
  
  # Add sample metadata
  seurat_obj$sample_id <- sample_id
  
  # Store the object in the list
  seurat_objects[[sample_id]] <- seurat_obj
}

# Merge all Seurat objects into one
ref <- merge(
  x = seurat_objects[[1]], 
  y = seurat_objects[-1], 
  add.cell.ids = names(seurat_objects)
)
ref=ref[,rownames(metadata)]
all(rownames(ref@meta.data)==rownames(metadata))
ref@meta.data=metadata
head(ref@meta.data)

,sampleAC,celltype_l2,celltype_l1.2
,<chr>,<chr>,<chr>
BDP1083_GAGACCCAGGTTAGTA-1,BDP1083_11_mut,Pod_Injured,Pod
BDP1083_TGTGGCGCATAGCCTT-1,BDP1083_11_mut,SMC,SMC
BDP1083_ATTGCATTCCGTTAGC-1,BDP1083_11_mut,PC,PC
BDP1083_ATTTAGCTCGAGGCGT-1,BDP1083_11_mut,PTS_Injured,PTS_Injured
BDP1083_ACTCAATTCACGGACA-1,BDP1083_11_mut,TAL,TAL
BDP1083_GTGAAACTCAACCAAA-1,BDP1083_11_mut,PTS1,PTS1


Loading sample: BDP1083 


Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


Loading sample: BDP1105 


Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


...

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


Loading sample: BDP1131 


Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


[1] TRUE

,sampleAC,celltype_l2,celltype_l1.2
,<chr>,<chr>,<chr>
BDP1083_GAGACCCAGGTTAGTA-1,BDP1083_11_mut,Pod_Injured,Pod
BDP1083_TGTGGCGCATAGCCTT-1,BDP1083_11_mut,SMC,SMC
BDP1083_ATTGCATTCCGTTAGC-1,BDP1083_11_mut,PC,PC
BDP1083_ATTTAGCTCGAGGCGT-1,BDP1083_11_mut,PTS_Injured,PTS_Injured
BDP1083_ACTCAATTCACGGACA-1,BDP1083_11_mut,TAL,TAL
BDP1083_GTGAAACTCAACCAAA-1,BDP1083_11_mut,PTS1,PTS1


In [11]:
# trace("LoadXenium",edit=TRUE)
LoadXenium=function (data.dir, fov = "fov", assay = "Xenium")
{
    data <- ReadXenium(data.dir = data.dir, type = c("centroids", 
        "segmentations"), )
    segmentations.data <- list(centroids = CreateCentroids(data$centroids),
        segmentation = CreateSegmentation(data$segmentations))
    coords <- CreateFOV(coords = segmentations.data, type = c("segmentation",
        "centroids"), molecules = data$microns, assay = assay)
    xenium.obj <- CreateSeuratObject(counts = data$matrix[["Gene Expression"]],
        assay = assay)
    xenium.obj[["BlankCodeword"]] <- CreateAssayObject(counts = data$matrix[["Unassigned Codeword"]])
    xenium.obj[["ControlCodeword"]] <- CreateAssayObject(counts = data$matrix[["Negative Control Codeword"]])
    xenium.obj[["ControlProbe"]] <- CreateAssayObject(counts = data$matrix[["Negative Control Probe"]])
    xenium.obj[[fov]] <- coords
    return(xenium.obj)
}

In [12]:
# Define the dictionary of dataset paths
dataset_paths <- list(
  '0027292__Region_1__20240530__125814' = "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027292__Region_2__20240530__125814' = "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027292__Region_3__20240530__125814' = "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027292__Region_4__20240530__125814' = "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027291__Region_1__20240530__125814' = "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027291__Region_2__20240530__125814' = "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027291__Region_3__20240530__125814' = "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027291__Region_4__20240530__125814' = "/data/lemsaraa/amina/ST/ourData/20240530__124752__A4413_ST014_X0069_X0070/output-XETG00046__",
  '0027119__Region_1__20240621__120943'= "/data/lemsaraa/amina/ST/ourData/20240621__120000__ST014_X0071_X0072_A4413/output-XETG00046__",
  '0027119__Region_2__20240621__120943'= "/data/lemsaraa/amina/ST/ourData/20240621__120000__ST014_X0071_X0072_A4413/output-XETG00046__",
  '0027120__Region_1__20240621__120943'="/data/lemsaraa/amina/ST/ourData/20240621__120000__ST014_X0071_X0072_A4413/output-XETG00046__",
  '0027120__Region_2__20240621__120943'= "/data/lemsaraa/amina/ST/ourData/20240621__120000__ST014_X0071_X0072_A4413/output-XETG00046__"
)


# LeftOut Genes Estimation

In [ ]:
# Generate 3 random sets of leftout genes
set.seed(42)  # For reproducibility
random_sets <- read.csv('data/random_added_genes.csv',row.names = 1)
results_list <- list()


# Loop over each random set of leftout genes
for (i in seq(3)) {
  leftout_genes <- random_sets[[i]]
#   print(leftout_genes)
  # Store results for this set
  results_set <- list()
  
  # Loop over each dataset in the dictionary
  for (dataset_name in names(dataset_paths)) {
    dataset_path <- dataset_paths[[dataset_name]]  # Get the path for the current dataset
    
    # Load Xenium object using the path from the dictionary
    xenium.ob <- LoadXenium(paste0(dataset_path,dataset_name), fov = "fov")
    mop_sub_sub <- xenium.ob@assays$Xenium@counts
    coord_sub_sub <- as.data.frame(xenium.ob@images$fov@boundaries$centroids@coords, 
                                   row.names = colnames(mop_sub_sub))
    
    # Exclude leftout genes
    mop_sub_sub <- mop_sub_sub[!(rownames(mop_sub_sub) %in% leftout_genes), ]
    
    # Run DOT workflow
    dot_srt <- setup.srt(srt_data = mop_sub_sub, srt_coords = coord_sub_sub)
    dot_ref <- setup.ref(ref_data = ref, ref_annotations = ref$celltype_l2)
    dot <- create.DOT(dot_srt, dot_ref)
    dot <- run.DOT.highresolution(dot, verbose = FALSE)
    cellt <- colnames(dot@weights)[apply(dot@weights, 1, which.max)]
  
    # Normalize data
    ref <- NormalizeData(ref)
    xenium.ob <- NormalizeData(xenium.ob)
    
    # Get common genes and compute predictions
    avr <- Seurat::AverageExpression(ref, group.by = "celltype_l2")$RNA
    avr <- avr[, colnames(dot@weights)]
      
    common_genes <- intersect(rownames(avr), rownames(xenium.ob))
    
    gene_pred <- avr %*% t(dot@weights)
    gene_org <- as.matrix(xenium.ob@assays$Xenium@data[common_genes,])
    gene_pred <- gene_pred[common_genes, colnames(mop_sub_sub)]
    
    # Compute correlation and cosine similarity
    result_gene <- corr_cos_plot(gene_org, gene_pred, sample = FALSE)
    correlation_results <- result_gene$correlation_results
    cosine_results <- result_gene$cosine_results
    
    # Save results for each gene in this dataset
    df=data.frame(
      gene = names(correlation_results),
      gorrelation = correlation_results,
      cosine = cosine_results
    )
    df= df%>% filter(gene %in% leftout_genes)
    results_set[[dataset_name]] <- df
  }
  
  # Store results for this random set
  results_list[[paste0("Set_", i)]] <- results_set
}

10X data contains more than one type and is being returned as a list containing matrices of each type.

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Computing spatial radius

Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”


Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.2 GiB”
Warning message in cor(X_org_, X_pred_, use = "pairwise.complete.obs"):
“the standard deviation is zero”
10X data contains more than one type and is being returned as a list containing matrices of each type.

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Computing spatial radius

Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning me

Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr,

Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr, con = out, size = 1L):
“problem writing to connection”
Warning message in writeBin(bfr,

Avoidable 3.989 seconds. This file is very unusual: it ends abruptly without a final newline, and also its size is a multiple of 4096 bytes. Please properly end the last row with a newline using for example 'echo >> file' to avoid this  time to copy.


Warning message in data.table::fread(file.path(data.dir, "transcripts.csv.gz")):
“Discarded single-line footer: <<281663956166173,"knjbokgi-1>>”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 1.6 GiB”
Computing spatial radius

Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did

Computing spatial radius

Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 472400)”


In [ ]:
combined_data1 <- do.call(rbind, lapply(names(results_list[['Set_1']]), function(sample_name) {
  df <- results_list[['Set_1']][[sample_name]]
  df$dataset <- sample_name
  return(df)
}))

combined_data2 <- do.call(rbind, lapply(names(results_list[['Set_2']]), function(sample_name) {
  df <- results_list[['Set_2']][[sample_name]]
  df$dataset <- sample_name
  return(df)
}))

combined_data3 <- do.call(rbind, lapply(names(results_list[['Set_3']]), function(sample_name) {
  df <- results_list[['Set_3']][[sample_name]]
  df$dataset <- sample_name
  return(df)
}))
combined_data=rbind(combined_data1,combined_data2,combined_data3)
write.csv(combined_data,'tab_out2/GEestimationRandom_dot.csv')